In [ ]:
'''
@Author: Haihui Pan
@Date: 2021-11-4
@Ref: https://pytorch.org/tutorials/beginner/basics/autogradqs_tutorial.html
'''

# AutoGrad自动微分

* 反向传播(back propagation,BP)是训练神经网络最常见的算法。在BP中，模型的参数会根据损失函数计算出来的梯度来进行更新。

* 为了计算梯度，PyTorch的***torch.autograd***提供了自动求导机制。

In [1]:
import torch
from torch.nn.functional import binary_cross_entropy_with_logits

#单层神经网络
x=torch.ones(5) #input
y=torch.zeros(3)
w=torch.randn(5,3,requires_grad=True)
b=torch.randn(3,requires_grad=True)
z=torch.matmul(x,w)+b

loss=binary_cross_entropy_with_logits(z, y)

注：你可以在创建Tensor时就设置requires_grad，也可以在创建之后通过调用x.requires_grad_(True)来进行设置

* 我们用来构造计算图的张量函数实际上是***Function***的对象。这个对象知道如何进行前馈计算，以及在反向传播过程中如何计算梯度。

In [2]:
#通过tensor.grad可以获取相关的反向传播函数
print(z.grad_fn)
print(loss.grad_fn)

## 计算梯度

* 为了优化神经网络的参数，我们需要计算Loss对于参数的梯度。为了计算梯度，我们可以调用***loss.backward()***,可以通过tensor.grad来获取对应的梯度

In [3]:
loss.backward()
print(w.grad)
print(b.grad)

tensor([[0.3173, 0.0152, 0.2986],
        [0.3173, 0.0152, 0.2986],
        [0.3173, 0.0152, 0.2986],
        [0.3173, 0.0152, 0.2986],
        [0.3173, 0.0152, 0.2986]])
tensor([0.3173, 0.0152, 0.2986])


In [4]:
print(x.requires_grad)

False


## 禁用自动梯度计算

* 默认情况先，将Tensor的requires_grad=True会导致Pytorch保留计算历史和支持梯度计算。例如，在测试场景下我们不需要去计算模型的梯度，我们只需要获取模型前馈计算的结果。通过torch.no_grad()模块，可以使得避免保存Tensor的计算历史。

* 在迁移学习中，我们可以需要去冻结某些参数，这时候需要将对应参数的requires_grad设置为false。
* torch.no_grad()可以加速前馈计算

In [5]:
z=torch.matmul(x,w)+b
print(z.requires_grad)

#不用保存计算历史
with torch.no_grad():
    z=torch.matmul(x,w)+b
    print(z.requires_grad)

True
False


In [6]:
#通过设置detach()，也可以达到相同效果
z=torch.matmul(x,w)+b
z_det=z.detach()
print(z_det.requires_grad)

False


## 计算图

In [8]:
%%html
<img src="autograd_01.png",width="40%",height="40%">

从概念上讲，在一个由函数组成的向无环图(DAG)中，autograd会保存Tensor和所有的操作。在DAG中，叶子是输入Tensor,根是输出Tensor。通过从根到叶跟踪这个图，可以使用链式法则自动计算梯度。

* 在前馈计算过程中，autograd同时做两件事:
  * 根据定义的操作计算对应的结果Tensor
  * 在DAG中维护操作的gradient function
  

* 当最终的输出Tensor（如计算得到的loss）调用.backward()时开始，autograd之后开始计算:
  * 计算每一个.grad_fn的梯度
  * 在.grad属性中累加Tensor的梯度
  * 利用链式法则，一直反向计算到叶子节点

注：PyTorch中的DAG是动态的，计算图是从头开始重新创建的。在每次调用.backward()之后，autograd开始填充新的图形。这允许你在模型中使用控制流语句，如果需要您可以在每次迭代中更改形状、大小和操作。

## Jacobian Products

* 在一些场景中，我们的loss输出为标量。但是，在一些场景下，我们的loss输出为一个向量,或者是其他张量。在这种情况中，PyTorch允许你去计算***Jacobian product***（并不是真正的梯度）。

* 给定函数$y=f(x),y=(y_{1},...,y_{m}),x=(x_{1},...,x_{n})$,则使用Jacobian矩阵来表示$y$对$x$的梯度：

$J=\bigl(\begin{smallmatrix}
 \frac{\partial y_{1}}{\partial x_{1}}& \cdots &  \frac{\partial y_{1}}{\partial x_{n}}\\ 
\vdots  & \ddots  & \vdots \\ 
 \frac{\partial y_{m}}{\partial x_{1}}& \cdots  & \frac{\partial y_{m}}{\partial x_{n}}
\end{smallmatrix}\bigr)$

* 在PyTorch中并不是计算Jacobian矩阵，而是计算$v^{T}J$，其中$v^{T}$是给定的input。只要在backward中传入参数$v$即可，其中$v$预原始的Tensor应该一致。

In [11]:
inp = torch.eye(5, requires_grad=True)
out=(inp+1).pow(2)

#计算梯度，当输入不是一个标量时需要传入一个与inp相同shape的单位向量，否则计算v^T*J时会出错
out.backward(torch.ones_like(inp),retain_graph=True)
#计算Jacobian porduct
print("First call\n", inp.grad)

#如果不清零梯度，则会进行梯度的累加
out.backward(torch.ones_like(inp),retain_graph=True)
print("Second call\n", inp.grad)

#如果不清零梯度，则会进行梯度的累加
out.backward(torch.ones_like(inp),retain_graph=True)
print("Third call\n", inp.grad)

First call
 tensor([[4., 2., 2., 2., 2.],
        [2., 4., 2., 2., 2.],
        [2., 2., 4., 2., 2.],
        [2., 2., 2., 4., 2.],
        [2., 2., 2., 2., 4.]])
Second call
 tensor([[8., 4., 4., 4., 4.],
        [4., 8., 4., 4., 4.],
        [4., 4., 8., 4., 4.],
        [4., 4., 4., 8., 4.],
        [4., 4., 4., 4., 8.]])
Third call
 tensor([[12.,  6.,  6.,  6.,  6.],
        [ 6., 12.,  6.,  6.,  6.],
        [ 6.,  6., 12.,  6.,  6.],
        [ 6.,  6.,  6., 12.,  6.],
        [ 6.,  6.,  6.,  6., 12.]])


注：当第二次使用相同的参数去调用backward时，对于inp的梯度结果是不一样的。这是由于在进行反向传播时，PyTorch会自动累加梯度，即将计算出来的梯度加上当前Tensor.grad上。如果你只需要去计算当前的梯度，则需要通过将Tensor的梯度设置为0。在实际应用中，optimizer.zero_grad()可以直接将所有的Tensor.grad清零。

In [13]:
#清零梯度，则会将先前累加的梯度设置为0
inp.grad.zero_()

out.backward(torch.ones_like(inp),retain_graph=True)
print("After zeroing grad\n",inp.grad)

After zeroing grad
 tensor([[4., 2., 2., 2., 2.],
        [2., 4., 2., 2., 2.],
        [2., 2., 4., 2., 2.],
        [2., 2., 2., 4., 2.],
        [2., 2., 2., 2., 4.]])
